### 텐서플로우를 이용하여 ML 구현

- 알고리즘 SVM 을 이용하여 BMI를 학습시키고, 비만도를 판정
- BMI 지수 = (몸무게/키)*키
- 데이터 3.bmi.csv

<img src='./텐서용어.png'>

In [ ]:
# 사용패키지 로드
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# 데이터로드
csv = pd.read_csv('./3_bmi.csv')
csv.shape

In [ ]:
csv.head(2)

In [ ]:
# 데이터 전처리
# height	weight의 최대값기준 정규화
csv['height'] = csv['height']  / csv['height'].max()
csv['weight'] = csv['weight']  / csv['weight'].max()

In [ ]:
csv['height'].min(), csv['height'].max()

In [ ]:
csv['weight'].min(), csv['weight'].max()

In [ ]:
csv.head(2)

In [ ]:
csv['label'].unique()

In [ ]:
# csv['label']의 백터화 => 분류변수
labelClass = {
    'thin':  [1,0,0],
    'normal':[0,1,0],
    'fat':   [0,0,1]
}

In [ ]:
# 데이터 전환 -> 값으로 배열을 넣는다 => np.array( 리스트 )
csv['label_bmi'] = csv['label'].apply( lambda x : np.array( labelClass[x] ) )
csv['label_bmi'][:2]

In [ ]:
csv.head(2)

In [ ]:
# 데이터 준비 -> 훈련용 데이터, 검증용 데이터로 분류
from sklearn.model_selection import train_test_split

In [ ]:
csv[ ['height','weight'] ].head(1), csv['label_bmi'][:1]

In [ ]:
# 일관성 있게 결과를 얻고 싶다면 => 난수의 시드를 고정해라
X_train, X_test, y_train, y_test = train_test_split(
    csv[ ['weight','height'] ], csv['label_bmi'],
    test_size = 0.25, random_state = 0 )

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_train[:2]

- 텐서플로우 등장
- 소프트맥스 회귀라는 알고리즘을 사용 -> DL 수행시 자주 사용하는 활성화 함수중에
  하나, softmax(), sigmod(),.. : 소프트맥스 회귀->인공신경망과 아주 유사
<img src='./sm0.png' width='200'>

- 각각의 입력값에 가중치(W)를 곱하고, 편향:바이어스(b)를 더하고 나서 
- softmax() 함수를 통과 시키면,  출력값이 나타나게 된다

<img src='./sm1.png' width='500'>

<img src='./sm2.png' width='500'>

<img src='./sm3.png' width='500'>

- **y = softmax( Wx + b )**
- 실제 텐서플로우로 구현시 차원유지를 위해서 Wx는 xW로 구현할수 있다 행렬곱

In [ ]:
# 소프트맥스회귀 식을 텐서플로우 구현하기 위해 구성요소 정의
# 입력 => x => 키, 몸무게  이렇것이  N개
#x = tf.placeholder( tf.float32, [None, 2] 
x  = tf.placeholder(tf.float32, [None, 2])

In [ ]:
# 출력  y_ -> 1,0,0 or 0,1,0 or 0,0,1 -> [None 3]
#y_ = tf.placeholder( tf.float32, [None, 3] )
y_ = tf.placeholder(tf.float32, [None, 3])

In [ ]:
# 가중치 : W -> 필터, 커널로 표현
# W*[None,2] + b = [None,3]
# [None,2]*[2, 3] + [3] = [None,3]
# tf.zeros() 통해서 해당 행렬에 기본값을 0으로 세팅
#W = tf.Variable( tf.zeros([2, 3]) )
W = tf.Variable(tf.zeros([2, 3]));

In [ ]:
# 편향, 바이어스
#b = tf.Variable( tf.zeros([3]) )
b = tf.Variable(tf.zeros([3]));    

In [ ]:
# 데이터 플로우 그래프 y
#y = tf.nn.softmax( tf.matmul( x, W ) + b )
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [ ]:
'%s x %s + %s = %s' % ( x.shape, W.shape, b.shape, y.shape ) # 출력형태

#### 학습

- 좋은 모델에 대한 정의
- 기준
    - 비용(cost), 손실(loss)등 원하는 결과에서 얼마나 떨어져 있는지 지표
    - 이런 격차를 줄이는 방향으로 내용 전개. 
    - 이를 위해서 통상 "크로스 엔트로피" 알고리즘 적용
    - 정보 이론 분야에 정보 알축 알고리즘으로 고안  
    - 도박, 머신러닝등에 중요한 아이디어로 사용
    <img src='./sm4.png' width='200'>

In [ ]:
# 텐서플로워로 크로스 엔트로피 구성
# y_ : 정답 레이블, y:예측 레이블
#cross_entropy = -tf.reduce_sum(y_ * tf.log(y) )
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

In [ ]:
# 크로스 엔트로피값을 최소화 하도록 작업이 진행
# 이런 지시는 => 경사 하강법(gradient descent algorithm)으로 수행
# 경사하강법을 적용하면, 텐서플로우가 각각의 변수를 비용을 줄이는 방향으로
# 조금씩 이동시켜서 연산. => 오차율이 최소가 되게 하기 위해 
# 경사 하강법 알고리즘 생성

# 하이퍼파라미터 0.01
#optimzer = tf.train.GradientDescentOptimizer( 0.01 ) 
optimizer     = tf.train.GradientDescentOptimizer(0.01)

In [ ]:
# 훈련 정의
#train = optimzer.minimize( cross_entropy )
train         = optimizer.minimize(cross_entropy)

In [ ]:
# 정답률 예측 정의
# y_ : 정답 레이블
# y  : 예측 레이블
# tf.argmax( y, 1 ) : 모델이 판단하기에 각 데이터별 가장 적합하다고 판단되는라벨
predict = tf.equal( tf.argmax( y, 1 ), tf.argmax( y_, 1) )

In [ ]:
# 정확도 정의
# predict => 불린형 리스트 => 부동소수 변환 => 평균
#accurary = tf.reduce_mean( tf.cast(predict, tf.float32) )
accuracy      = tf.reduce_mean(tf.cast(predict, tf.float32))

- 세션 가동 => 실제 연산 수행 => 결과 획득

In [ ]:
# 구동을 반복해서 시키면 학습량이 증가해서, 정확도가 상승한다
with tf.Session() as sess:
    # 변수 초기화
    sess.run( tf.global_variables_initializer() )
    print( "W", W)
    print( "b", b)
    
    # 학습 데이터 0 ~ 15000 => 100개 단위로 학습하겟다 => 150회 학습
    TERM = 100
    size = int(X_train.shape[0]/TERM)
    print( "b", size)
    
    # 학습
    # 성능 향상(일반적)
    for step in range( size*16 ):
        # X_train에서 데이터를 추출 ->  시작위치
        startIdx = (int)(step * TERM/16)
        # 앞에서 부터 TERM  단위로 훈련 데이터를 추출 하겟다
        rows = X_train[ startIdx :startIdx + TERM ]
    
        # 학습 데이터 구성
        x_pat = rows[ ['weight','height'] ]
        y_ans = list(y_train[ startIdx :startIdx + TERM ])
        # 평가를 위한 구조
        fd    = { x:x_pat,  y_:y_ans }
        
        # 학습 세트 훈련
        # sess.run 구동시 데이터를 전달 => feed_dict=
        sess.run( train, feed_dict=fd )
        
        # 중간 확인
        if step % 16 == 0:# 15번째 마다 점검
            # 크로스 엔트로피값 획득
            cre = sess.run( cross_entropy, feed_dict=fd )
            # 정확도 : X_train, X_test, y_train, y_test
            acc = sess.run( accuracy, feed_dict={x:X_test, y_:list(y_test)} )
            print( 'step=%s cre=%s acc=%s' % (step, cre, acc) )
            
    # 최종적인 정답값 확인
    acc = sess.run( accuracy, feed_dict={x:X_test, y_:list(y_test)} )
    print('정답률', acc)
    
    # 텐서들의 관계및 흐름을 보기 위해 지원: 텐서보드
    tf.summary.FileWriter( './bmi_tf_log', graph=sess.graph )

In [ ]:
# 텐서보드 가동
# shell을 가동 
!tensorboard --logdir=bmi_tf_log
# http://ip:6006

In [ ]:
# 컴퓨터 따라서 실행이 않될수 있으니 
$ tensroboard --logdir=bmi_tf_log